In [1]:
# import pakietów
import pandas as pd
import urllib
from IPython.display import HTML
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import re
from statistics import mean

In [2]:
#import danych
col_names = ['ID', 'gender', 'city', 'income', 'birth_date', 'application_date', 'requested_amount', 
             'requested_period', 'financial_obligations', 'employer_name', 'account_bank', 'mobile_verification_flag',
             'var_5', 'var_1', 'offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation',
             'filled_form_flag', 'device', 'var_2', 'source', 'var_4', 'logged_in_flag', 'disbursed_flag']
dataset = pd.read_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/train.csv', delimiter = ',', dtype='str', 
                      header = None, names = col_names, engine='python', index_col = False, skiprows = 1 )

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000,5,0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000,2,0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000,4,0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000,5,0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000,2,25000,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0


In [3]:
months_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
               'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

# TO DO - opis funckji i testy jednostkowe
def date_transform(date:str, twentieth_cent=False):
    day = date[:2]
    month = months_dict[date[3:6]]
    if twentieth_cent:
        return '19' + date[-2:] + "-" + month + "-" + day
    else:
        return '20' + date[-2:] + "-" + month + "-" + day

# formatowanie danych
dataset['income'] = dataset['income'].astype(float)
dataset['birth_date'] = dataset.apply(lambda x: date_transform(x['birth_date'], True), axis=1)
dataset['application_date'] = dataset.apply(lambda x: date_transform(x['application_date']), axis=1)
dataset['requested_amount'] = dataset['requested_amount'].astype(float)
dataset['financial_obligations'] = dataset['financial_obligations'].astype(float)

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag
0,ID000002C20,Female,Delhi,20000.0,1978-05-23,2015-05-15,300000.0,5,0.0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000.0,1985-10-07,2015-05-04,200000.0,2,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500.0,1981-10-10,2015-05-19,600000.0,4,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000.0,1987-11-30,2015-05-09,1000000.0,5,0.0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000.0,1984-02-17,2015-05-20,500000.0,2,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0


In [4]:
#city - wikipedia
cities = set(dataset['city'])
cities_list = []
latitudes = []
latitudes_NS = []
longitudes = []
longitudes_WE = []

for city in cities:
    try:
        with urlopen("https://en.wikipedia.org/wiki/" + city) as page:
            soup = BeautifulSoup(page.read(), "html.parser")
            response_list = soup.select(".geo-dec")
            if response_list:
                city_coords = soup.select(".geo-dec")[0].get_text()
                if '°' in city_coords:
                    city_coords = city_coords.split(' ')[0].split('°') + city_coords.split(' ')[1].split('°')
                    cities_list.append(city)
                    latitudes.append(city_coords[0])
                    latitudes_NS.append(city_coords[1])
                    longitudes.append(city_coords[2])
                    longitudes_WE.append(city_coords[3])
    except urllib.request.HTTPError:
        pass
        
wiki_cities_coords = pd.DataFrame()
wiki_cities_coords['city'] = pd.Series(cities_list)
wiki_cities_coords['latitude'] = pd.Series(latitudes)
wiki_cities_coords['latitude_NS'] = pd.Series(latitudes_NS)
wiki_cities_coords['longitude'] = pd.Series(longitudes)
wiki_cities_coords['longitude_WE'] = pd.Series(longitudes_WE)

wiki_cities_coords.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/wiki_cities_coords.csv', index=False)

In [5]:
cities_left = list(cities - set(wiki_cities_coords['city']))
len(cities_left)

162

In [6]:
#city - from google by selenium


def split_google_coords(str):
    city = str.split(',')[0]
    str = str.split('\n')
    longitude, latitude, dms_lat, dms_long = None, None, None, None
    for item in str:
        if 'longitude ' in item:
            longitude = item.split('longitude')[1]
        if 'latitude ' in item:
            latitude = item.split('latitude')[1]
        if 'dms lat ' in item:
            dms_lat = item.split('dms lat ')[1]
        if 'dms long ' in item:
            dms_long = item.split('dms long ')[1]
    return [city, longitude, latitude, dms_lat, dms_long]


google_cities = []
another_iteration = True

while another_iteration:
    driver = webdriver.Chrome('chromedriver')
    cities_done = []
    for city in cities_left:
        driver.get('https://www.google.pl/search?q=' + city + '+latitude+and+longitude+in+decimal')
        selector = 'div.mod'
        try:
            data_string = driver.find_element_by_css_selector(selector).text.lower()
            coords = split_google_coords(data_string)
            if 'india' in  data_string:
                google_cities.append([city, coords])
                cities_done.append(city)
        except NoSuchElementException:
            pass
    driver.close()
    if len(cities_done) > 0:
        cities_left = list(set(cities_left) - set(cities_done))
        print("Zostało: " + str(len(cities_left)))
    else:
        another_iteration = False
    
google_cities_coords = pd.DataFrame()
google_cities_coords['city'] = pd.Series([item[0] for item in google_cities])
google_cities_coords['coords_string'] = pd.Series([item[1] for item in google_cities])

google_cities_coords.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/google_cities_coords.csv', index=False)

Zostało: 124
Zostało: 111
Zostało: 96
Zostało: 83
Zostało: 81


In [31]:
#city - from google by selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

google_cities = []
another_iteration = True

while another_iteration:
    driver = webdriver.Chrome('chromedriver')
    cities_done = []
    for city in cities_left:
        driver.get('https://www.google.pl/search?q=' + city + '+india+gps+coords')
        selector = 'h3'
        try:
            links = driver.find_elements_by_css_selector(selector)
            links = [(item, item.text.lower()) for item in links]
            links = [item for item in links if 'latitude:' in item[1] or 'longitude:' in item[1]]
            if links:
                links[0][0].click()
                undone = True
                while undone:
                    try:
                        if 'google' not in driver.current_url:
                            data = driver.page_source.lower().split("<")
                            data = [item for item in data if 'latitude:' in item or 'longitude:' in item]
                            undone = False
                    except NoSuchElementException:
                        pass
                google_cities.append([city, data])
            cities_done.append(city)
        except NoSuchElementException:
            pass
    driver.close()
    if len(cities_done) > 0:
        cities_left = list(set(cities_left) - set(cities_done))
        print("Zostało: " + str(len(cities_left)))
    else:
        another_iteration = False
    
google_cities_coords = pd.DataFrame()
google_cities_coords['city'] = pd.Series([item[0] for item in google_cities])
google_cities_coords['coords_string'] = pd.Series([item[1] for item in google_cities])

google_cities_coords.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/google_cities_coords2.csv', index=False)

Zostało: 0


In [27]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://www.google.pl/search?q=Gaya+india+gps+coords')
links = driver.find_elements_by_css_selector("h3")
links = [(item, item.text.lower()) for item in links if 'latitude:' in item.text.lower() or 'longitude:' in item.text.lower()]
links[0][0].click()
undone = True
while undone:
    try:
        if 'google' not in driver.current_url:
            data = driver.page_source.split("<")
            data = [item.lower() for item in data if 'latitude:' in item.lower() or 'longitude:' in item.lower()]
            undone = False
    except NoSuchElementException:
        pass
driver.close()
data    

['title>gps coordinates of gaya, india. latitude: 24.7969 longitude: 85.0039',
 'meta name="dcterms.title" content="gps coordinates of gaya, india. latitude: 24.7969 longitude: 85.0039">\n',
 'meta property="og:title" content="gps coordinates of gaya, india. latitude: 24.7969 longitude: 85.0039">\n',
 'div class="addthis_sharing_toolbox addthis_32x32_style" data-url="https://latitude.to/map/in/india/cities/gaya" data-title="gps coordinates of gaya, india. latitude: 24.7969 longitude: 85.0039" style="clear: both;">\n  ',
 'b>latitude:',
 'b>longitude:']

In [7]:
cities = set(dataset['city'])

def split_google_coords(str):
    city = str.split(',')[0]
    str = str.split('\n')
    longitude, latitude, dms_lat, dms_long = None, None, None, None
    for item in str:
        if 'longitude ' in item:
            longitude = item.split('longitude')[1]
        if 'latitude ' in item:
            latitude = item.split('latitude')[1]
        if 'dms lat ' in item:
            dms_lat = item.split('dms lat ')[1]
        if 'dms long ' in item:
            dms_long = item.split('dms long ')[1]
    return [city, longitude, latitude, dms_lat, dms_long]

    
driver = webdriver.Chrome('chromedriver')
cities_done = []
for city in list(cities):
    page_wiki = ''
    page_google = ''
    page_google_first_link = ''
    
    #wikipedia
    try:
        with urlopen("https://en.wikipedia.org/wiki/" + city) as page:
            soup = BeautifulSoup(page.read(), "html.parser")
            response_list = soup.select(".geo-dec")
            if response_list:
                page_wiki = soup.select(".geo-dec")[0].get_text().lower()
    except urllib.request.HTTPError:
        pass
    
    #google
    driver.get('https://www.google.pl/search?q=' + city + '+latitude+and+longitude+in+decimal')
    selector = 'div.mod'
    try:
        result = driver.find_element_by_css_selector(selector).text.lower()
        if 'india' in  result:
            page_google = split_google_coords(result)
    except NoSuchElementException:
        pass
    
    #google first returned link
    driver.get('https://www.google.pl/search?q=' + city + '+india+gps+coords')
    selector = 'h3'
    try:
        links = driver.find_elements_by_css_selector(selector)
        links = [(item, item.text.lower()) for item in links]
        links = [item for item in links if 'latitude:' in item[1] or 'longitude:' in item[1]]
        if links:
            links[0][0].click()
            undone = True
            while undone:
                if 'google' not in driver.current_url:
                    try:
                        page_google_first_link = driver.find_element_by_name('DD').get_attribute('value')
                    except NoSuchElementException:
                        pass                        
                    undone = False
    except NoSuchElementException:
        pass
    cities_done.append([city, page_wiki, page_google, page_google_first_link])
    
driver.close()
cities_coords_raw_data = pd.DataFrame(cities_done, columns = ['city', 'page_wiki', 'page_google', 'page_google_first_link'])
cities_coords_raw_data.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/all_coords.csv', index=False)

In [63]:
def wiki2coords(text):
    if text:
        coords = text.split(' ')
        coords[0] = float(coords[0].split('°')[0])
        coords[1] = float(coords[1].split('°')[0])
        return [coords[0], coords[1]]
    else:
        return [None, None]
       
wiki = list(cities_coords_raw_data['page_wiki'])
wiki = [wiki2coords(item) for item in wiki]

In [65]:
google = list(cities_coords_raw_data['page_google'])
pattern = "[0-9]+° [0-9]+' [0-9]+.[0-9]+'' [nsew]"

def google2coords(list):
    if list is not None:
        results = []
        for item in list:
            if item is not None:
                found = re.findall(pattern, item)
                results += found
        n, e = None, None
        if results:
            results = [(int(x.split('°')[0]), int(x.split(' ')[1].split("'")[0]), float(x.split(' ')[2].split("'")[0]), x.split(' ')[3]) for x in results]
            n = [x[0] + x[1]/60.0 + x[2]/3600.0 for x in results if x[3]=='n']
            e = [x[0] + x[1]/60.0 + x[2]/3600.0 for x in results if x[3]=='e']
            n = mean(n) if n else None
            e = mean(e) if e else None
        return [n, e]
    else:
        return None

google = [google2coords(item) for item in google]

In [71]:
google_first_link = list(cities_coords_raw_data['page_google_first_link'])
google_first_link = [ [float(x.split(' ')[0]), float(x.split(' ')[1])] if x else [None, None] for x in google_first_link]